# This notebook contains functions for generating the plots defined in the Plots_Class notebook

## Importing libraries and modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp

## Setting the size of the tick labels

In [ ]:
plt.rc('xtick',labelsize = 15)
plt.rc('ytick',labelsize = 15)

# Plot Functions

## Function for generating $H_B$ vs $H_P$ plots

In [ ]:
def HBvsHP(bio,power,S,
           subst_flat,
           gibbs_flat,
           tradeoff,
           PH = False):
    """
     Arguments:
     
     bio: 3d np.array containing the survival diversity (Shannon index) 
          for each instance of standard Gibbs energy, number of habitats 
          and width of the distribution
     
     power: 3d np.array (the same shape as bio) containing the diversity of power supply (Shannon index) 
          for each instance of standard Gibbs energy, number of habitats 
          and width of the distribution
     
     S: 1d np.array containing all the instances of widths for the 
        distribution of parameter values. S has shape either (1,) or (4,)
     
     subst_flat: boolean. True if the distribution for the input substrate is flat. 
                          False, otherwise
     
     gibbs_flat: boolean. True if the distribution for the standard Gibbs energy of reactions 
                          is flat. False, otherwise
     
     tradeoff: boolean. True if the disrtibutions for uptake rates and maintenance 
                        powers are both NOT flat. False, otherwise.
     
     PH: boolean. OPTIONAL
               If True, the plots include the pH. The default is False
                        
    
    Returns: The scatter plot of all the pairs of values (H_B,H_P) for each instance of distribution width 
    """
    
    HB = bio.reshape((-1,len(S))) 
    HP = power.reshape((-1,len(S))) 
    # HB[:,s] contains all the values in bio corresponding to the width instance s 
    # HP[:,s] contains all the values in power corresponding to the width instance s 
    # in the same order as that of HB[:,s]
    Colors = []
    for i in range(HB.shape[0]):
        ind = int(i/bio.shape[1])
        Colors.append([1-ind/bio.shape[0],ind/bio.shape[0],0.2])
        
    MS = 8 # Marker Size
    if (subst_flat & gibbs_flat & (not(tradeoff))):
        plt.scatter(HP[:,0],HB[:,0],s=MS,c=Colors,marker='.')
        plt.xlabel('$H_P$',fontsize=15)
        plt.ylabel('$H_B$',fontsize=15)
        if PH:
            plt.title('$H_B$ vs $H_P$ along a pH-gradient, $s$='+'{:.3f}'.format(S[0]))
        else:
            plt.title('$H_B$ vs $H_P$, $s$='+'{:.3f}'.format(S[0]))
        plt.show() 
    else:
        if len(S)==1:
            plt.scatter(HP[:,0],HB[:,0],s=MS,c=Colors,marker='.')
            plt.xlabel('$H_P$',fontsize=15)
            plt.ylabel('$H_B$',fontsize=15)
            if PH:
                plt.title('$H_B$ vs $H_P$ along a pH-gradient, $s$='+'{:.3f}'.format(S[0]))
            else:
                plt.title('$H_B$ vs $H_P$, $s$='+'{:.3f}'.format(S[0]))
            plt.show() 
        else:
            fig,ax = plt.subplots(2,2)
            for i in range(len(S)):
                ax[int(i/2),i-2*int(i/2)].scatter(HP[:,i],HB[:,i],s=MS,c=Colors,marker='.')
                ax[int(i/2),i-2*int(i/2)].set_xlabel('$H_P$',fontsize=15)
                ax[int(i/2),i-2*int(i/2)].set_ylabel('$H_B$',fontsize=15)
                if PH:
                    ax[int(i/2),i-2*int(i/2)].set_title('pH, $s$='+'{:.3f}'.format(S[i]))
                else:
                    ax[int(i/2),i-2*int(i/2)].set_title('$s$='+'{:.3f}'.format(S[i]))
            for a in ax.flat:
                a.label_outer() 
            
            plt.show() 
    return None 



## PH plots

In [ ]:
def pH_plots(data):
    
    """
    Generates the plots for survival probability vs energy scale
    when the pH is considered
    
    Aruments:
    ----------
    
    data : tuple 
    
    """
    
    LS = 15 #label size
    pH, abundance, bio_ent, pow_ent, E, signs = data 
    N = abundance.shape[1]
    fig, ax = plt.subplots(2,2,sharex=True)
    for l in range(N):
        if signs[l] == 1:
            ax[0,0].plot(pH,abundance[:,l],'r-',linewidth=2)
            ax[1,0].plot(pH,E[:,l],'r-',linewidth=2)
        else:
            ax[0,0].plot(pH,abundance[:,l],'b-',linewidth=2)
            ax[1,0].plot(pH,E[:,l],'b-',linewidth=2)
    
    ax[0,1].plot(pH,bio_ent,'r-',linewidth=3)
    ax[1,1].plot(pH,pow_ent,'r-',linewidth=3)
    
    ax[0,0].set_xlim([0,14])
    ax[0,1].set_xlim([0,14])
    ax[1,0].set_xlim([0,14])
    ax[1,1].set_xlim([0,14])
    
    ax[0,0].set_ylabel('$c_i^*\,\mathrm{(cell/cm^3)}$',fontsize=LS)
    ax[0,1].set_ylabel('$H_B$',fontsize=LS)
    ax[1,0].set_ylabel('$-E_{eq}/\Delta G^0_r$',fontsize=LS)
    ax[1,1].set_ylabel('$H_P$',fontsize=LS)
    ax[1,0].set_xlabel('$pH$',fontsize=LS)
    ax[1,1].set_xlabel('$pH$',fontsize=LS)
    
    ax[0,0].tick_params(labelsize=10,labelbottom=False)
    ax[0,1].tick_params(labelsize=10,labelbottom=False,
                        labelleft=False,labelright=True)
    ax[1,0].tick_params(labelsize=10)
    ax[1,1].tick_params(labelsize=10,labelleft=False,labelright=True)
    
        
    plt.show() 
    
    return None 

# Model Parameter profiles

## Uptake rate, $r^i_{max}$, and power demand, $P_{0i}$, profiles:

In [ ]:
def rp_profiles(data, P0 = 1, rmax = 1):
    """
    Arguments:
    
    data: tuple containing the elements
          (x,n,s,tradeoff)
          or
          (x,n,s,tradeoff,combined)
          where
          x: np.array. Contains the values for the x-axis
          n: int. Number of species
          s: float. Width of the distribution
          tradeoff: boolean. True if the distributions 
                             for r and p are NOT flat.
                             False, otherwise
          combined: boolean. True if all the parameters follow
                             non-flat distributions. False, otherwise
    Returns: 
    
    r: np.array containing the values for the uptake parameters
    p: np.array containing the values fpr the maintenance power
    """
    
    comb = False
    if len(data) == 4:
        x,n,s,tradeoff = data
    else:
        x,n,s,tradeoff,comb = data
    
    if comb:
        r = np.exp(-(x-n/2)**2/(s**2*n**2/2))+1/100
        p = np.exp(-(x-n/2)**2/(s**2*n**2/2))+1/20
    else:
        if tradeoff:
            r = np.exp(-(x-n/2)**2/(s**2*n**2))+1/100
            p = np.exp(-(x-n/2)**2/(s**2*n**2))+1/20
        else:
            r = np.ones(n)
            p = np.ones(n)
    
    return rmax*r,P0*p



## Input substrate concentration profile: $S_{0i}$

In [ ]:
def subst_profile(x,n,s,
                subst_flat,
                S0 = 1):
    
    """
    Arguments:
    
    x: np.array. Contains the values for the x-axis
    n: int. Number of species
    s: float. Width of the distribution
    subst_flat: boolean. True if the distribution 
                       for the input substrate concentration is flat.
                       False, otherwise
    Returns: 
    
    S: np.array containing the values for the input substrate concentrations
    """
    
    if subst_flat:
        S = 1.2*np.ones(n)
    else:
        S = 10**3*np.exp(-(x-n/2)**2/(s**2*n**2))+1.2
            
    return S0*S



## Reaction Gibbs energy profile: $\Delta G^r_{0i}$

In [ ]:
def gibbs_profile(data,E0=1):
    """
    Arguments:
    data: tuple containing the elements
          (x,n,s,gibs_flat)
          or
          (x,n,s,gibs_flat,cimbined)
          where
          
          x: np.array. Contains the values for the x-axis
          n: int. Number of species
          s: float. Width of the distribution
          gibbs_flat: boolean. True if the distribution 
                               for the Gibbs energy of reactions is flat.
                               False, otherwise
          combined: boolean. True if all the parameters follow
                             non-flat distributions. False, otherwise
    Returns: 
    
    E: np.array containing the values for the Gibbs energy of reactions
    """
    comb = False
    if len(data) == 4:
        x,s,n,gibbs_flat = data 
    else:
        x,s,n,gibbs_flat,comb = data
        
    if comb:
        E = np.exp(-(x-n/2)**2/(5*s**2*n**2)) + 1/6
    else:
        if gibbs_flat:
            E = np.ones(n)
        else:
            E = np.exp(-(x-n/2)**2/(s**2*n**2)) + 1/6
    
    return E0*E 



# Lamber W-function 

In [ ]:
def LW(x,y):
    """
    Arguments:
    
    x,y: np.arrays of the same size
    
    Returns: 
    np.array : $W(x e^y)$ 
               If y + ln(x) >= 5, the approximation to $W(z)$
               derived in the Supplementary Information is employed
    """
    
    if not(isinstance(x,np.ndarray)):
        x = np.array([x])
    if not(isinstance(y,np.ndarray)):
        y = np.array([y])
    Z = y + np.log(x)
    index = Z>=5
    comp_index = np.logical_not(index)
    lw = np.ones(len(x))
    Z0 = Z[index]
    h = Z0 - np.log(Z0)
    lw[index] = h - h*(np.log(h) - np.log(Z0))/(1+h)
    Z1 = Z[comp_index]
    lw[comp_index] = np.real(sp.lambertw(np.exp(Z1)))
    return lw


# Shannon Index (entropy)

In [ ]:
def H(P):
    
    """
    Arguments:
    
    P: np.array containing non-negative entries and such that 
       \sum_i P_i = 1
    
    Returns: 
    
     The Shannon entropy of the distribution P
    """
    
    P = P[P>0]
    return -sum(P*(np.log(P))) 
